<h1 align="center">Exercício de Aula</h1>
           
 * Daniel Silva, up201503212
 * João Conde, up201503256
 * Miguel Ramalho, up201402027
 
<span style="float:right">11/04/2019</span>

| Partida  | Chegada   | Tipo        | Custo | Procura |
|----------|-----------|-------------|-------|---------|
| Porto    |  New York |  Early bird | 178 € | 63      |
| Porto    |  Chicago  |  Early bird | 268 € | 84      |
| Porto    |  Orlando  |  Early bird | 228 € | 86      |
| Porto    |  New York |  Full price | 380 € | 31      |
| Porto    |  Chicago  |  Full price | 456 € | 12      |
| Porto    |  Orlando  |  Full price | 560 € | 21      |
| Lisbon   |  New York |  Early bird | 199 € | 50      |
| Lisbon   |  Chicago  |  Early bird | 249 € | 107     |
| Lisbon   |  Orlando  |  Early bird | 349 € | 75      |
| Lisbon   |  New York |  Full price | 385 € | 29      |
| Lisbon   |  Chicago  |  Full price | 444 € | 14      |
| Lisbon   |  Orlando  |  Full price | 580 € | 18      |
| New York |  Chicago  |  Early bird | 179 € | 122     |
| New York |  Chicago  |  Full price | 380 € | 16      |
| New York |  Orlando  |  Early bird | 224 € | 88      |
| New York |  Orlando  |  Full price | 582 € | 19      |

### Install modules

In [1]:
!pip install ortools

### Import
 * Import [Google Or-tools](https://developers.google.com/optimization/)
 * Jupyter HTML display

In [2]:
from ortools.linear_solver import pywraplp
from IPython.core.display import display, HTML

In [3]:
# create a solver instance for TAP
solver = pywraplp.Solver('tap_solver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Decision Variables
For each flight and for each fare there is a variable. 

This decision variable describes the number of seats to sell for each flight and fare.

`8 vars x 2 fares = 16 decision variables`

### Flights From Porto

In [4]:
pne = solver.IntVar(0, 63, 'Porto->New York, Early Bird')
pnf = solver.IntVar(0, 31, 'Porto->New York, Full Price')

In [5]:
poe = solver.IntVar(0, 86, 'Porto->Orlando, Early Bird')
pof = solver.IntVar(0, 21, 'Porto->Orlando, Full Price')

In [6]:
pce = solver.IntVar(0, 84, 'Porto->Chicago, Early Bird')
pcf = solver.IntVar(0, 12, 'Porto->Chicago, Full Price')

### Flights From Lisbon

In [7]:
lne = solver.IntVar(0, 50, 'Lisbon->New York, Early Bird')
lnf = solver.IntVar(0, 29, 'Lisbon->New York, Full Price')

In [8]:
loe = solver.IntVar(0, 75, 'Lisbon->Orlando, Early Bird')
lof = solver.IntVar(0, 18, 'Lisbon->Orlando, Full Price')

In [9]:
lce = solver.IntVar(0, 107, 'Lisbon->Chicago, Early Bird')
lcf = solver.IntVar(0, 14,  'Lisbon->Chicago, Full Price')

### Flights From New York

In [10]:
noe = solver.IntVar(0, 88, 'New York->Orlando, Early Bird')
nof = solver.IntVar(0, 19, 'New York->Orlando, Full Price')

In [11]:
nce = solver.IntVar(0, 122, 'New York->Chicago, Early Bird')
ncf = solver.IntVar(0, 16,  'New York->Chicago, Full Price')

# Objective Function
Maximize income

In [12]:
solver.Maximize(pne*178 + pnf*380 + lne*199 + lnf*385 + pce*268 + pcf*456 + poe*228 + pof*560 + lce*249 + lcf*444 + loe*349 + lof*580 + noe*224 + nof*582 + nce*179 + ncf*380)

# Constraints

In [13]:
PC = 274 # PLANE CAPACITY

These constraints ensure that each of the sources of the network can send `PC` travelers and that each of the sinks can receive `PC`

In [14]:
# Flights that can leave Porto 
solver.Add(pne + pnf + poe + pof + pce + pcf <= PC);

In [15]:
# Flights that can leave Lisbon
solver.Add(lne + lnf + loe + lof + lce + lcf <= PC);

In [16]:
# Max arriving at Orlando 
solver.Add(poe + pof + loe + lof + noe + nof <= PC);

In [17]:
# Max arriving at Chicago
solver.Add(pce + pcf + lce + lcf + nce + ncf<= PC);

The following constraints are not necessary, but are interesting in that they describe the flow of travelers in the `NY` station:

In [18]:
# Max arriving at NY
solver.Add(pne + pnf + lne + lnf <= 2 * PC);

In [19]:
# Max leaving from NY
solver.Add(noe + nof + nce + ncf <= (2 * PC) - (poe + pof + loe + lof + pce + pcf + lce + lcf));

# Solution

In [20]:
print('Number of variables   =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

Number of variables   = 16
Number of constraints = 6


In [21]:
result_status = solver.Solve()

In [22]:
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

In [23]:
display(HTML('<h1 align="center">Optimal Value = %d€</h1>' % solver.Objective().Value()))

In [24]:
decision_variables = [pne,pce,poe,pnf,pcf,pof,lne,lce,loe,lnf,lcf,lof,nce,ncf,noe,nof]
for v in decision_variables:
    print('%35s = %3d seats' % (v.name(), v.solution_value()))

        Porto->New York, Early Bird =  63 seats
         Porto->Chicago, Early Bird =  84 seats
         Porto->Orlando, Early Bird =  63 seats
        Porto->New York, Full Price =  31 seats
         Porto->Chicago, Full Price =  12 seats
         Porto->Orlando, Full Price =  21 seats
       Lisbon->New York, Early Bird =  50 seats
        Lisbon->Chicago, Early Bird =  88 seats
        Lisbon->Orlando, Early Bird =  75 seats
       Lisbon->New York, Full Price =  29 seats
        Lisbon->Chicago, Full Price =  14 seats
        Lisbon->Orlando, Full Price =  18 seats
      New York->Chicago, Early Bird =  60 seats
      New York->Chicago, Full Price =  16 seats
      New York->Orlando, Early Bird =  78 seats
      New York->Orlando, Full Price =  19 seats
